In [6]:
using GeometryTypes
using LCMGL
using Meshing
using DrakeVisualizer
using BenchmarkTools

In [137]:
module Gjk

import GeometryTypes
import StaticArrays: SVector, @SVector, MVector, @MVector
const gt = GeometryTypes

immutable Tagged{P}
    point::P
    tag::Int
end

type AcceleratedMesh{N, T, MeshType <: gt.AbstractMesh}
    mesh::MeshType
    neighbors::Vector{Set{Int}}
end

function plane_fit(data)
    centroid = mean(data, 2)
    U, s, V = svd(data .- centroid)
    i = indmin(s)
    normal = U[:,i]
    offset = dot(normal, centroid)
    normal, offset
end
    

function AcceleratedMesh{N, T}(mesh::gt.AbstractMesh{gt.Point{N, T}})
    neighbors = Set{Int}[Set{Int}() for vertex in gt.vertices(mesh)]
    for face in gt.faces(mesh)
        for i in 1:length(face)
            for j in i+1:length(face)
                if face[i] != face[j]
                    push!(neighbors[gt.onebased(face, i)], gt.onebased(face, j))
                    push!(neighbors[gt.onebased(face, j)], gt.onebased(face, i))
                end
            end
        end
    end
    
    # The enhanced GJK algorithm is susceptible to becoming stuck in local minima
    # if all of the neighbors of a given vertex are coplanar. It also benefits from
    # having some distant neighbors for each node, to avoid having to always take the
    # long way around the mesh to get to the other side. 
    # To try to fix this, we will compute a fitting plane for all of the existing
    # neighbors for each vertex. We will then add neighbors corresponding to the 
    # vertices at the maximum distance on each side of that plane. 
    verts = gt.vertices(mesh)
    for i in eachindex(neighbors)
        @assert length(neighbors[i]) >= 2
        normal, offset = plane_fit(reinterpret(T, 
            [verts[n] for n in neighbors[i]], (N, length(neighbors[i]))))
        push!(neighbors[i], indmin(map(v -> dot(convert(gt.Point{N, T}, normal), v), verts)))
        push!(neighbors[i], indmax(map(v -> dot(convert(gt.Point{N, T}, normal), v), verts)))
#         push!(neighbors[i], rand(1:length(verts)))
    end
    AcceleratedMesh{N, T, typeof(mesh)}(mesh, neighbors)
end

any_inside{N, T}(mesh::AcceleratedMesh{N, T}) = Tagged(convert(gt.Vec{N, T}, gt.vertices(mesh.mesh)[1]), 1)
any_inside(geometry) = Tagged(gt.any_inside(geometry), 0)
any_inside{N, T}(mesh::gt.AbstractMesh{gt.Point{N, T}}) = Tagged(convert(gt.Vec{N, T}, gt.vertices(mesh)[1]), 0)

function support_vector_max(geometry, direction, initial_guess::Tagged)
    best_pt, score = gt.support_vector_max(geometry, direction)
    Tagged(best_pt, 0), score
end
    
function support_vector_max{N, T}(mesh::AcceleratedMesh{N, T}, direction, 
    initial_guess::Tagged{gt.Vec{N, T}})
    best = initial_guess
    score = dot(direction, best.point)
    verts = gt.vertices(mesh.mesh)
    while true
        candidates = mesh.neighbors[best.tag]
#         @show best score candidates map(n -> dot(direction, verts[n]), candidates)
        neighbor_index, best_neighbor_score = gt.argmax(n -> dot(direction, verts[n]), candidates)
        if best_neighbor_score > score || (best_neighbor_score == score && neighbor_index > best.tag)
            score = best_neighbor_score
            best = Tagged(convert(gt.Vec{N, T}, verts[neighbor_index]), neighbor_index)
        else
            break
        end
    end
    best, score
end 

function support_vector_max{N, T}(mesh::gt.HomogenousMesh{gt.Point{N, T}}, direction, initial_guess::Tagged) 
    best_arg, best_value = gt.argmax(x-> x⋅direction, gt.vertices(mesh))
    Tagged(convert(gt.Vec{N, T}, best_arg), 0), best_value
end

typealias Simplex{M, N, T} MVector{M, Tagged{gt.Vec{N, T}}}

function gjk{M, N, T}(hull, simplex::Simplex{M, N, T},
        max_iter=100, atol=1e-6)
#     typealias SimplexView SubArray{Tagged{gt.Vec{N, T}}, 1, Simplex, Tuple{UnitRange{Int}}, true}
    
    zero_point = zero(gt.Vec{N, T})
    tagged_pt_best = any_inside(hull)
    pt_best::gt.Vec{N, T} = tagged_pt_best.point
    simplex_points = 1
    for k in 1:max_iter
        direction = -pt_best
        # Do a linear search over just the simplex to find a good starting point,
        # then do a neighbor-wise search over the mesh to try to find an even
        # better point.
        starting_vertex, _ = gt.argmax(t -> dot(t.point, direction), simplex)
        improved_vertex::Tagged{gt.Vec{N, T}}, score::T = support_vector_max(hull, direction, starting_vertex)
        # If we found something no better than the existing simplex, then return
        if score <= dot(pt_best, direction) + atol
            break
        else
            if simplex_points <= N
                simplex[simplex_points + 1] = improved_vertex
                simplex_points += 1
            else
                worst_index, _ = gt.argmax(i -> -dot(simplex[i].point, direction), 1:simplex_points)
                simplex[worst_index] = improved_vertex
            end
            pt_best, sqd::T = gt.proj_sqdist(zero_point, gt.Simplex([convert(gt.Vec{N, T}, t.point) for t in view(simplex, 1:simplex_points)]...))
            sqd == 0 && return simplex
        end

    end
    simplex[1:simplex_points]
end

function gjk{N}(::Type{Val{N}}, hull, max_iter=100, atol=1e-6)
    start = any_inside(hull)
    gjk(hull, (@MVector [start for i in 1:$(N + 1)]), max_iter, atol)
#     gjk(hull, Simplex{N, T}, max_iter, atol)
end

# function neighbors{T}(mesh::gt.HomogenousMesh{gt.Point{3, T}}, vertex_index::Integer)
#     neighboring_faces = filter(face -> any(gt.onebased(face, i) == vertex_index for i in 1:length(face)), gt.faces(mesh))
#     Tagged{gt.Point{3, T}}[Tagged(gt.vertices(mesh)[face][i], gt.onebased(face, i)) for face in neighboring_faces for i in 1:length(face) if gt.onebased(face, i) != vertex_index]
# end

end

Gjk

In [138]:
s = Simplex(Vec(2., -1), Vec(2.5, 0), Vec(2., 1), Vec(1., 1), Vec(1., -1))
Gjk.gjk(Val{2}, s, 10)

LoadError: LoadError: eval cannot be used in a generated function
while loading In[138], in expression starting on line 2

In [126]:
@code_warntype Gjk.gjk(Val{2}, Val{3}, Float64, s, 10, 1e-6)

Variables:
  #self#::Gjk.#gjk
  #temp#@_2::Type{Val{2}}
  #temp#@_3::Type{Val{3}}
  #temp#@_4::Type{Float64}
  hull::GeometryTypes.Simplex{5,FixedSizeArrays.Vec{2,Float64}}
  max_iter::Int64
  atol::Float64
  Simplex::Type{StaticArrays.MVector{3,Gjk.Tagged{FixedSizeArrays.Vec{2,Float64}}}}
  zero_point::FixedSizeArrays.Vec{2,Float64}
  tagged_pt_best::Gjk.Tagged{FixedSizeArrays.Vec{2,Float64}}
  pt_best::FixedSizeArrays.Vec{2,Float64}
  simplex::CORE.BOX
  #13::Gjk.##13#17{Gjk.Tagged{FixedSizeArrays.Vec{2,Float64}}}
  simplex_points::Int64
  #temp#@_15::Int64
  #14::Gjk.##14#18
  #15::Gjk.##15#19{DataType}
  #16::Gjk.##16#20{2,Float64}
  k::Int64
  direction::CORE.BOX
  #temp#@_21::Int64
  starting_vertex::Gjk.Tagged{FixedSizeArrays.Vec{2,Float64}}
  _::ANY
  #temp#@_24::Int64
  improved_vertex::Gjk.Tagged{FixedSizeArrays.Vec{2,Float64}}
  score::Float64
  #temp#@_27::Int64
  worst_index::Int64
  #temp#@_29::ANY
  sqd::Float64
  rvalue::FixedSizeArrays.Vec{2,Float64}
  #temp#@_32::Bool

In [127]:
mesh = DrakeVisualizer.contour_mesh(x -> sum((x - [2, -1.5, 0]).^2) - 1, [-1.5, -1.5, -1.5], [1.5, 1.5, 1.5])
simplex = Gjk.gjk(Val{3}, Float64, mesh)
Visualizer(mesh)
LCMGLClient("simplex") do lcmgl
    color(lcmgl, 1, 0, 0, 0.5)
    for tagged in simplex
        sphere(lcmgl, [tagged.point...], 0.02, 20, 20)
    end
    color(lcmgl, 0, 0, 1, 0.5)
    sphere(lcmgl, [0., 0, 0], 0.02, 20, 20)
    switch_buffer(lcmgl)
end

LoadError: LoadError: MethodError: no method matching gjk(::Type{Val{3}}, ::Type{Float64}, ::GeometryTypes.HomogenousMesh{FixedSizeArrays.Point{3,Float64},GeometryTypes.Face{3,Int64,0},Void,Void,Void,Void,Void})
Closest candidates are:
  gjk{N,NPlus1,T}(::Type{Val{N}}, !Matched::Type{Val{NPlus1}}, !Matched::Type{T}, !Matched::Any) at In[124]:94
  gjk{N,NPlus1,T}(::Type{Val{N}}, !Matched::Type{Val{NPlus1}}, !Matched::Type{T}, !Matched::Any, !Matched::Any) at In[124]:94
  gjk{N,NPlus1,T}(::Type{Val{N}}, !Matched::Type{Val{NPlus1}}, !Matched::Type{T}, !Matched::Any, !Matched::Any, !Matched::Any) at In[124]:94
while loading In[127], in expression starting on line 2

In [23]:
acc = Gjk.AcceleratedMesh(mesh)
simplex = Gjk.gjk(Val{3}, Float64, acc)
Visualizer(mesh)
LCMGLClient("simplex") do lcmgl
    color(lcmgl, 1, 0, 0, 0.5)
    for tagged in simplex
        sphere(lcmgl, [tagged.point...], 0.02, 20, 20)
    end
    color(lcmgl, 0, 0, 1, 0.5)
    sphere(lcmgl, [0., 0, 0], 0.02, 20, 20)
    switch_buffer(lcmgl)
end

In [24]:
@benchmark Gjk.gjk(Val{3}, Float64, $mesh)

BenchmarkTools.Trial: 
  samples:          10000
  evals/sample:     1
  time tolerance:   5.00%
  memory tolerance: 1.00%
  memory estimate:  8.08 kb
  allocs estimate:  219
  minimum time:     52.10 μs (0.00% GC)
  median time:      61.74 μs (0.00% GC)
  mean time:        62.80 μs (1.56% GC)
  maximum time:     3.69 ms (95.84% GC)

In [25]:
@benchmark Gjk.gjk(Val{3}, Float64, $acc)

BenchmarkTools.Trial: 
  samples:          10000
  evals/sample:     1
  time tolerance:   5.00%
  memory tolerance: 1.00%
  memory estimate:  10.86 kb
  allocs estimate:  251
  minimum time:     54.27 μs (0.00% GC)
  median time:      63.63 μs (0.00% GC)
  mean time:        65.91 μs (1.74% GC)
  maximum time:     3.34 ms (95.15% GC)